### RL Agent for self-driven car in GTA - SA

In [1]:
# Import dependencies
import numpy as np
from PIL import ImageGrab
import cv2
import time
from numpy import ones,vstack
from numpy.linalg import lstsq
from directkeys import PressKey, W, A, S, D
from statistics import mean

In [2]:
def draw_lanes(img, lines, color=[0, 255, 255], thickness=3):

    # if this fails, go with some default line
    try:

        # finds the maximum y value for a lane marker 
        # (since we cannot assume the horizon will always be at the same point.)

        ys = []  
        for i in lines:
            for ii in i:
                ys += [ii[1],ii[3]]
        min_y = min(ys) #Highest point. Draw the line
        max_y = 456
        line_dict = {}

        for idx,i in enumerate(lines):
            for xyxy in i:
                # These four lines:
                # modified from http://stackoverflow.com/questions/21565994/method-to-return-the-equation-of-a-straight-line-given-two-points
                # Used to calculate the definition of a line, given two sets of coords.
                x_coords = (xyxy[0],xyxy[2])
                y_coords = (xyxy[1],xyxy[3])
                A = vstack([x_coords,ones(len(x_coords))]).T
                m, b = lstsq(A, y_coords)[0]

                # Calculating our new, and improved, xs
                x1 = (min_y-b) / m
                x2 = (max_y-b) / m
                # Store slope, bias, actual x and y
                line_dict[idx] = [m,b,[int(x1), min_y, int(x2), max_y]]
                
        final_lanes = {}

        for idx in line_dict:
            final_lanes_copy = final_lanes.copy()
            m = line_dict[idx][0]
            b = line_dict[idx][1]
            line = line_dict[idx][2]
            
            if len(final_lanes) == 0:
                final_lanes[m] = [ [m,b,line] ]
                
            else:
                found_copy = False

                for other_ms in final_lanes_copy:

                    if not found_copy:
                        # Slope and bias of 10% -> same line
                        if abs(other_ms*1.2) > abs(m) > abs(other_ms*0.8):
                            if abs(final_lanes_copy[other_ms][0][1]*1.2) > abs(b) > abs(final_lanes_copy[other_ms][0][1]*0.8):
                                final_lanes[other_ms].append([m,b,line])
                                found_copy = True
                                break
                        else:
                            final_lanes[m] = [ [m,b,line] ] #Two most common lines -> lanes

        line_counter = {}

        for lanes in final_lanes:
            line_counter[lanes] = len(final_lanes[lanes])

        top_lanes = sorted(line_counter.items(), key=lambda item: item[1])[::-1][:2]

        lane1_id = top_lanes[0][0]
        lane2_id = top_lanes[1][0]

        def average_lane(lane_data):
            x1s = []
            y1s = []
            x2s = []
            y2s = []
            for data in lane_data:
                x1s.append(data[2][0])
                y1s.append(data[2][1])
                x2s.append(data[2][2])
                y2s.append(data[2][3])
                # return avarage
            return int(mean(x1s)), int(mean(y1s)), int(mean(x2s)), int(mean(y2s)) 

        l1_x1, l1_y1, l1_x2, l1_y2 = average_lane(final_lanes[lane1_id])
        l2_x1, l2_y1, l2_x2, l2_y2 = average_lane(final_lanes[lane2_id])

        return [l1_x1, l1_y1, l1_x2, l1_y2], [l2_x1, l2_y1, l2_x2, l2_y2]
    except Exception as e:
        print(str(e))

In [4]:
def roi(img, vertices):
    #blank mask:
    mask = np.zeros_like(img)
    # fill the mask
    cv2.fillPoly(mask, vertices, 255)
    # now only show the area that is the mask
    masked = cv2.bitwise_and(img, mask)
    return masked

def process_img(image):
    original_image = image
    # convert to gray
    processed_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # edge detection
    processed_img =  cv2.Canny(processed_img, threshold1 = 100, threshold2=200)
    processed_img = cv2.GaussianBlur(processed_img,(3,3),1) # 3x3 kernel, 0-> sigmaX,Y calculated from kernel
    vertices = np.array([[10,486],[10,327],[166,168],[479,168],[635,327],[635,486]], np.int32)
    processed_img = roi(processed_img, [vertices])
    lines = cv2.HoughLinesP(processed_img, 1, np.pi/180, 180, np.array([]), 100, 5)
    try:
        l1, l2 = draw_lanes(original_image,lines)
        cv2.line(original_image, (l1[0], l1[1]), (l1[2], l1[3]), [0,255,0], 30)
        cv2.line(original_image, (l2[0], l2[1]), (l2[2], l2[3]), [0,255,0], 30)
    except Exception as e:
        print(str(e))
        pass
    
    try:
        for coords in lines:
            coords = coords[0]
            try:
                cv2.line(processed_img, (coords[0], coords[1]), (coords[2], coords[3]), [255,0,0], 3)  
            except Exception as e:
                print('Hey{}'.format(str(e)))
    except Exception as e:
        pass

    return processed_img,original_image

def main(): 
    for i in range(4):
        print(i)
    print('********************LET THE GAMES BEGIN********************')
    last_time = time.time()
    while(True):
        # 800x600 windowed mode
        screen =  np.array(ImageGrab.grab(bbox=(10,30,635,486)))
#         print('loop took {} seconds'.format(time.time()-last_time))
        last_time = time.time()
        new_screen,original_image = process_img(screen)
        cv2.imshow('window',new_screen)
        cv2.imshow('window2',cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(5) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

In [5]:
main()

0
1
2
3
********************LET THE GAMES BEGIN********************


c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\envs\reinforceenv\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Python int too large to convert to C long
cannot convert float infinity to integer
'NoneType' object is not iterable


c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\envs\reinforceenv\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in double_scalars
c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\envs\reinforceenv\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in double_scalars


cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infi

cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infi

c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\envs\reinforceenv\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


cannot convert float NaN to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
list index out of range
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
Python int too large to convert to C long
canno

cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
cannot convert float infinity to integer
'NoneType' object is not iterable
